# How to simulate the LUCJ ansatz using matrix product states

Following from the previous guide, we now show how to use ffsim to simulate the [LUCJ ansatz](../explanations/lucj.ipynb) using matrix product states. In this way, we can calculate an approximation to the LUCJ energy, which is itself an approximation to the ground state energy, for an ethene molecule. This is particularly useful in complicated cases, such as for large molecules, where even the LUCJ energy cannot be computed exactly. 

As before, let's start by building the molecule.

In [1]:
import pyscf
import pyscf.mcscf

import ffsim

# Build an ethene molecule
bond_distance = 1.339
a = 0.5 * bond_distance
b = a + 0.5626
c = 0.9289
mol = pyscf.gto.Mole()
mol.build(
    atom=[
        ["C", (0, 0, a)],
        ["C", (0, 0, -a)],
        ["H", (0, c, b)],
        ["H", (0, -c, b)],
        ["H", (0, c, -b)],
        ["H", (0, -c, -b)],
    ],
    basis="sto-6g",
    symmetry="d2h",
)

# Define active space
active_space = range(mol.nelectron // 2 - 2, mol.nelectron // 2 + 2)

# Get molecular data and molecular Hamiltonian (one- and two-body tensors)
scf = pyscf.scf.RHF(mol).run()
mol_data = ffsim.MolecularData.from_scf(scf, active_space=active_space)
norb = mol_data.norb
nelec = mol_data.nelec
mol_hamiltonian = mol_data.hamiltonian

# Compute FCI energy
mol_data.run_fci()

print(f"norb = {norb}")
print(f"nelec = {nelec}")

converged SCF energy = -77.8266321248744
Parsing /tmp/tmp08hsp77h
converged SCF energy = -77.8266321248744
CASCI E = -77.8742165643862  E(CI) = -4.02122442107773  S^2 = 0.0000000
norb = 4
nelec = (2, 2)


Overwritten attributes  get_ovlp get_hcore  of <class 'pyscf.scf.hf.RHF'>
/home/bart/PycharmProjects/ffsim/.ffsim_dev/lib/python3.12/site-packages/pyscf/gto/mole.py:1294: UserWarning: Function mol.dumps drops attribute energy_nuc because it is not JSON-serializable
  warnings.warn(msg)
/home/bart/PycharmProjects/ffsim/.ffsim_dev/lib/python3.12/site-packages/pyscf/gto/mole.py:1294: UserWarning: Function mol.dumps drops attribute intor_symmetric because it is not JSON-serializable
  warnings.warn(msg)


Since our molecule has a closed-shell Hartree-Fock state, we'll use the spin-balanced variant of the UCJ ansatz, [UCJOpSpinBalanced](../api/ffsim.rst#ffsim.UCJOpSpinBalanced). We'll initialize the ansatz from t2 amplitudes obtained from a CCSD calculation and we'll restrict same-spin interactions to a line topology, and opposite-spin interactions to those within the same spatial orbital, which allows the ansatz to be simulated directly on a square lattice.

The following code cell initializes the LUCJ ansatz operator.

In [2]:
from pyscf import cc

# Get CCSD t2 amplitudes for initializing the ansatz
ccsd = cc.CCSD(
    scf,
    frozen=[i for i in range(mol.nao_nr()) if i not in active_space],
).run()

# Construct LUCJ operator
n_reps = 1
pairs_aa = [(p, p + 1) for p in range(norb - 1)]
pairs_ab = [(p, p) for p in range(norb)]
interaction_pairs = (pairs_aa, pairs_ab)

lucj_operator = ffsim.UCJOpSpinBalanced.from_t_amplitudes(
    ccsd.t2, n_reps=n_reps, interaction_pairs=interaction_pairs
)

E(CCSD) = -77.87421536374029  E_corr = -0.04758323886585589


<class 'pyscf.cc.ccsd.CCSD'> does not have attributes  converged


## Convert the Hamiltonian to a matrix product operator (MPO)

Currently, our Hamiltonian is an instance of the `MolecularHamiltonian` class. Using the `from_molecular_hamiltonian` method from the `MolecularHamiltonianMPOModel` class, we can convert this to a TeNPy `MPOModel`, which respects the fermionic symmetries. We can then construct the MPO using the `H_MPO` attribute and use this `MPO` object as outlined in the [TeNPy MPO documentation](https://tenpy.readthedocs.io/en/latest/reference/tenpy.networks.mpo.MPO.html#tenpy.networks.mpo.MPO). For example, the `MPO` class attribute `chi` tells us the MPO bond dimension, which is an important indicator of how complicated the Hamiltonian is in an MPO representation.

In [3]:
from ffsim.tenpy.hamiltonians.molecular_hamiltonian import MolecularHamiltonianMPOModel

print("original Hamiltonian type = ", type(mol_hamiltonian))
hamiltonian_mpo_model = MolecularHamiltonianMPOModel.from_molecular_hamiltonian(
    mol_hamiltonian
)
hamiltonian_mpo = hamiltonian_mpo_model.H_MPO
print("converted Hamiltonian type = ", type(hamiltonian_mpo))
print("maximum MPO bond dimension = ", max(hamiltonian_mpo.chi))

original Hamiltonian type =  <class 'ffsim.hamiltonians.molecular_hamiltonian.MolecularHamiltonian'>


AttributeError: 'MolecularHamiltonianMPOModel' object has no attribute 'H_MPO'

Optionally, we can pass the `decimal_places` argument to the `from_molecular_hamiltonian` method, which rounds the precision of the input one-body and two-body tensors. This reduces the MPO bond dimension at the expense of simulation accuracy.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MaxNLocator

dp_list = np.arange(1, 16, dtype=int)
chi_list = []
for dp in dp_list:
    hamiltonian_mpo_model = MolecularHamiltonianMPOModel.from_molecular_hamiltonian(
        mol_hamiltonian, decimal_places=dp
    )
    hamiltonian_mpo = hamiltonian_mpo_model.H_MPO
    chi_list.append(max(hamiltonian_mpo.chi))

fig = plt.figure()
ax = plt.subplot(111)
ax.plot(dp_list, chi_list, ".-")
ax.set_xlabel("precision of one-body and two-body tensors / decimal places")
ax.set_ylabel("maximum MPO bond dimension")
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.grid(visible=True)
plt.show()

## Construct the LUCJ circuit as a matrix product state (MPS)

Our wavefunction ansatz operator, on the other hand, is an instance of the `UCJOpSpinBalanced` class. In a future guide, we will show in detail how we can use such an ansatz to build and transpile Qiskit quantum circuits. In this guide, we will use this ansatz operator to construct our wavefunction as a TeNPy MPS, which respects the fermionic symmetries. Behind the scenes, this executes the ansatz as a fermionic circuit using the TEBD algorithm. 

We can pass the `options` dictionary and `norm_tol` to the `lucj_circuit_as_mps` function to control the accuracy of our MPS approximation. The `options` parameter is detailed in the [TeNPy TEBDEngine documentation](https://tenpy.readthedocs.io/en/latest/reference/tenpy.algorithms.tebd.TEBDEngine.html#tenpy.algorithms.tebd.TEBDEngine). The `norm_tol` parameter is defined in other contexts in the TeNPy library, e.g. in the [TeNPy DMRGEngine documentation](https://tenpy.readthedocs.io/en/latest/reference/tenpy.algorithms.dmrg.DMRGEngine.html#cfg-option-DMRGEngine.norm_tol). The most relevant key for us in the `options` dictionary is `trunc_params`, which defines the truncation parameters for our quantum circuit. In particular, `chi_max` sets the maximum bond dimension, and `svd_min` sets the minimum Schmidt value cutoff. We also introduce the `norm_tol` parameter, which sets the maximum norm error above which the wavefunction is recanonicalized.

In addition to the wavefunction as an MPS, the `lucj_circuit_as_mps` function also returns `chi_list`, which is a list of MPS bond dimensions that is stored after each two-site gate is applied to our initial Hartree-Fock state. This gives us an indication of how the entanglement grows in the system as we run our circuit. In the example below, we set the maximum allowed bond dimension to 15, and after running the circuit, we can see that the maximum bond dimension reaches 15. This indicates that we have most likely truncated the bond dimension with our choice of `chi_max`.

In [ ]:
from ffsim.tenpy.circuits.lucj_circuit import lucj_circuit_as_mps

options = {"trunc_params": {"chi_max": 15, "svd_min": 1e-6}}
psi_mps, chi_list = lucj_circuit_as_mps(
    norb, nelec, lucj_operator, options, norm_tol=1e-5
)
print("wavefunction type = ", type(psi_mps))
print(psi_mps)
print("maximum MPS bond dimension = ", np.max(chi_list))

## Compare the energies

Now that we have converted our `MolecularHamilonian` to an MPO, and our LUCJ ansatz to an MPS, we can contract the tensors to compute the energy. In order of increasing accuracy, we can compare the LUCJ (MPS) energy, the LUCJ energy, and the FCI energy.

In [ ]:
import numpy as np
from qiskit.circuit import QuantumCircuit, QuantumRegister

# Compute the LUCJ (MPS) energy
lucj_mps_energy = hamiltonian_mpo.expectation_value_finite(psi_mps)
print("LUCJ (MPS) energy = ", lucj_mps_energy)

# Compute the LUCJ energy
qubits = QuantumRegister(2 * norb)
circuit = QuantumCircuit(qubits)
circuit.append(ffsim.qiskit.PrepareHartreeFockJW(norb, nelec), qubits)
circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(lucj_operator), qubits)
lucj_state = ffsim.qiskit.final_state_vector(circuit).vec
hamiltonian = ffsim.linear_operator(mol_hamiltonian, norb=norb, nelec=nelec)
lucj_energy = np.real(np.vdot(lucj_state, hamiltonian @ lucj_state))
print("LUCJ energy = ", lucj_energy)

# Print the FCI energy
fci_energy = mol_data.fci_energy
print("FCI energy = ", fci_energy)

To illustrate the effects of the truncation parameters more clearly, we can plot the energies at different values of `svd_min` and `chi_max`.

In [ ]:
import matplotlib.gridspec as gridspec

svd_min_list = [1e-3, 1e-6]
chi_max_list = np.arange(2, 21, 2)
lucj_mps_energy = np.zeros((2, len(chi_max_list)))
max_chi = np.zeros((2, len(chi_max_list)))

for i, svd_min in enumerate(svd_min_list):
    options["trunc_params"]["svd_min"] = svd_min
    for j, chi_max in enumerate(chi_max_list):
        options["trunc_params"]["chi_max"] = int(chi_max)
        psi_mps, chi_list = lucj_circuit_as_mps(
            norb, nelec, lucj_operator, options, norm_tol=1e-5
        )
        lucj_mps_energy[i, j] = hamiltonian_mpo.expectation_value_finite(psi_mps)
        max_chi[i, j] = np.max(chi_list)

fig = plt.figure(figsize=(10, 4))
gs = gridspec.GridSpec(1, 2, wspace=0.3)
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])

for i in [0, 1]:
    ax0.plot(
        chi_max_list,
        lucj_mps_energy[i, :],
        ".-",
        label=f"$\\lambda_\\text{{min}}=10^{{{np.log10(svd_min_list[i]):g}}}$",
    )
    ax0.axvline(
        x=np.max(max_chi[i, :]),
        c=f"C{i}",
        linestyle="dashed",
        label=f"$\\chi_\\text{{max}}(10^{{{np.log10(svd_min_list[i]):g}}})$",
    )

ax0.set_xlabel("maximum MPS bond dimension")
ax0.set_ylabel("$E$")
ax0.xaxis.set_major_locator(MaxNLocator(integer=True))
ax0.axhline(y=lucj_energy, color="k", linestyle="dashed", label="$E_\\text{LUCJ}$")
ax0.axhline(y=fci_energy, color="k", linestyle="dotted", label="$E_\\text{FCI}$")
ax0.legend(loc="best")

for i in [0, 1]:
    ax1.plot(
        chi_max_list,
        np.abs(np.subtract(lucj_mps_energy[i, :], lucj_energy)),
        ".-",
        label=f"$\\lambda_\\text{{min}}=10^{{{np.log10(svd_min_list[i]):g}}}$",
    )
    ax1.axvline(
        x=np.max(max_chi[i, :]),
        c=f"C{i}",
        linestyle="dashed",
        label=f"$\\chi_\\text{{max}}(10^{{{np.log10(svd_min_list[i]):g}}})$",
    )
ax1.set_xlabel("maximum MPS bond dimension")
ax1.set_ylabel("$|E-E_\\text{LUCJ}|$")
ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
ax1.set_yscale("log")
ax1.legend(loc="best")

plt.show()

From the above plots, we can see that above an MPS bond dimension of 16, the MPS representation of the LUCJ circuit is exact.